# Smash Bros

Today's extra nerdy dataset is stats from a tournament of the game *Super Smash Bros. Melee* for the Nintendo GameCube.  The stats were recorded during the tournament [Smash Summit 2017](https://liquipedia.net/smash/Smash_Summit/Spring_2017), these stats were collected by an older version the [Slippi](https://slippi.gg/faq) project.  The data was collected with this [R script](https://github.com/AdamSpannbauer/twitch_chat/blob/master/r_scripts/slippi_win.R) in 2017 (no promises the script still works).

The data contains a player id column, `gamerTag`, and a binary variable indicating if the player won the set: `won`.  The rest of the variables are different statistics counted up throughout the set. 

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


data_url = "https://raw.githubusercontent.com/AdamSpannbauer/twitch_chat/master/data/slippi_data/ssbm_win.csv"
ssbm = pd.read_csv(data_url)

<IPython.core.display.Javascript object>

In most machine learning contexts, we will isolate our input features from our target variable.  Player ID won't be much use since we want our model to predict wins for every player, not just the 16 players in our data set.

* So we'll drop `gamerTag` & `won` and assign it to `X`
    * This `X` is a conventional way to specify that this variable is our input features
* It's also conventional to put your target into a variable named `y`

In [3]:
# Store every column but 'gamerTag' and 'won' in X
# What you're making predictions with
X = ssbm.drop(columns=["gamerTag", "won"])

# Store what you want to predict ('won') in y
# What you're trying to predict
y = ssbm["won"]

<IPython.core.display.Javascript object>

#### Scenario 1: 

You're teaching a person about your favorite subject.  How do you prove that they've learned something?


#### Scenario 2:

Someone from last semester wrote down all the answers to the final on a napkin and gave them to you.  You look down and it reads: `['A', 'B', 'A', 'D', 'C', 'A', 'D']`.  They claim they got 100% correct on the test with these answers.  Do you trust that they'll work for you? Why? or Why not?


#### Scenario 3:

Someone from last semester wrote down all the answers to the final on a napkin and gave them to you.  You look down and it reads: `['Churchill', 'Industrial Revolution', 'Oprah', 'Clark Kent', 'Coolio', 'Adam', 'Donatello']`.  They claim they got 100% correct on the test with these answers.  Do you trust that they'll work for you? Why? or Why not?  Is this any different from how you feel about Scenario 2?


#### Scenario 4:

Someone from last semester wrote down all the answers to the final on a napkin and gave them to you.  You look down and it reads:

```python
{
    'Name a british person': 'Churchill', 
    'Name a word that starts with I and a word that starts with R': 'Industrial Revolution',
    'Who has the network "OWN"?': 'Oprah', 
    "What's Superman's real name": 'Clark Kent', 
    'Name a ninja turle': 'Donatello'
}
```
They claim they got 100% correct on the test with these questions & answers.  Do you trust that they'll work for you? Why? or Why not?  Is this any different from how you feel about Scenario 3?


#### Scenario 5:

You studied for the final extremely well.  You don't think you're an expert in the field (i.e. you know you can't answer questions on topics you've never seen before, but you feel you could make a good guess).  You feel that you can accurately answer a variety of questions on the subject matter you've covered.

----

* Perform a train test split

In [4]:
rows, columns = X.shape
rows

56

<IPython.core.display.Javascript object>

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

<IPython.core.display.Javascript object>

In [6]:
rows, cols = X_train.shape
rows

39

<IPython.core.display.Javascript object>

In [7]:
rows, cols = X_test.shape
rows

17

<IPython.core.display.Javascript object>

* Use SelectKBest to choose two features to be our predictors

In [8]:
# x is numeric
X.head(1)

,numCombosStarted,totalComboDamage,averageComboDamage,maxComboDamage,percentTimeClosestCenter,percentTimeInShield,rollCount,numRecoveries,totalRecoveryDamage,averageRecoveryDamage,...,recoverySuccessPercent,numEdgeguards,numFailedEdgeguards,edgeguardSuccessPercent,numPunishesStarted,numKillingPunishes,totalPunishDamage,averagePunishDamage,neutralWinPercent,openingsPerKill
0,79,1119.17,14.166709,37.92,53.205737,7.801918,43,23,375.85,16.341304,...,47.826087,24,23,4.166667,66,9,1124.17,17.032879,45.833333,7.333333


<IPython.core.display.Javascript object>

In [9]:
# y is cat
y.head(1)

0    0.0
Name: won, dtype: float64

<IPython.core.display.Javascript object>

In [10]:
# based on datatypes and ANOVA (f score) could help id good vars
# stats.f_oneway(group1['won'], group2['won'])

selector = SelectKBest(f_classif, k=2)

selector.fit(X_train, y_train)

best_X_train = selector.transform(X_train)
best_X_test = selector.transform(X_test)

<IPython.core.display.Javascript object>

* Fit a model to predict the target variable

In [11]:
# Classification model type
model = LogisticRegression()
model.fit(best_X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

<IPython.core.display.Javascript object>

* Score the model on the training data

In [12]:
model.score(best_X_train, y_train)

0.8461538461538461

<IPython.core.display.Javascript object>

* Score the model on the testing data

In [13]:
model.score(best_X_test, y_test)

0.7058823529411765

<IPython.core.display.Javascript object>